In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, chdir
from os.path import join, basename

sys.path.append(str(Path(getcwd()).parent))
current_dir = getcwd()
print(current_dir)

In [ ]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)

In [ ]:
from bulletin import default_input, default_output, hoje, ontem
from bulletin.systems import CasosConfirmados, Notifica
from bulletin.utils.normalize import normalize_text

In [ ]:
cc = CasosConfirmados()

In [ ]:
nt = Notifica()
nt.load('notifica', compress=False)
nt.df.set_index('id',inplace=True)
nt.df = nt.df.loc[((nt.df['classificacao_final']==2)&(nt.df['excluir_ficha']==2)&(nt.df['status_notificacao'].isin([1,2])))]

In [ ]:
fator_risco = nt.df[nt.schema.loc[nt.schema['group_name']=='fator_risco','column']].copy()
fator_risco

In [ ]:
gestantes = fator_risco[['gestante','periodo_gestacao','gestante_alto_risco']]
fator_risco = fator_risco.drop(columns=gestantes.columns.tolist() + ['outras_morbidades'])

In [ ]:
fator_risco = fator_risco.sort_index(axis=1)

In [ ]:
fator_risco['fatores'] = fator_risco.progress_apply(lambda x: x.loc[x==1].index, axis=1)

In [ ]:
fator_risco['fatores_len'] = fator_risco['fatores'].apply(len)
fator_risco['fatores_str'] = fator_risco['fatores'].apply(lambda x: ", ".join(x))

In [ ]:
fator_risco = fator_risco.join(nt.df[['sexo']])

In [ ]:
sexo = fator_risco.groupby(['fatores_str','fatores_len','sexo'])[['fatores']].count().unstack(-1).fillna(0)

In [ ]:
sexo.sort_index(axis=0,level='fatores_len')